In [87]:
import pandas as pd
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary

In [88]:
train_df = pd.read_csv('../dataset/preprocessed_train.csv')
test_df = pd.read_csv('../dataset/preprocessed_test.csv')

In [89]:
corpus = pd.concat([train_df.comment_text, test_df.comment_text]).reset_index(drop=True).dropna().astype(str)

In [90]:
corpus_split = corpus.str.split()

In [91]:
from gensim import corpora, models

dictionary = corpora.Dictionary(corpus_split)

In [92]:
corpus_BoW = [dictionary.doc2bow(text) for text in corpus_split]


In [93]:
lda = LdaModel(corpus_BoW, id2word=dictionary, num_topics=50)

In [94]:
import pyLDAvis

In [95]:
topics = {topic_id: words for (topic_id, words) in lda.print_topics(num_topics=50)}

In [96]:
topics.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [106]:
for topic, p in lda.get_document_topics(training_BoW[0]):
    print(topics[topic])
    print()

0.059*"cock" + 0.029*"film" + 0.028*"sucks" + 0.026*"man" + 0.017*"season" + 0.016*"hit" + 0.016*"episode" + 0.015*"king" + 0.015*"cawk" + 0.015*"movie"

0.062*"help" + 0.036*"thank" + 0.034*"welcome" + 0.030*"pages" + 0.023*"hello" + 0.023*"ask" + 0.022*"hope" + 0.021*"articles" + 0.019*"editing" + 0.017*"question"

0.040*"dont" + 0.035*"im" + 0.023*"good" + 0.016*"ive" + 0.015*"thats" + 0.015*"got" + 0.015*"time" + 0.014*"right" + 0.014*"going" + 0.013*"sorry"

0.059*"deletion" + 0.046*"deleted" + 0.032*"articles" + 0.031*"ahole" + 0.031*"notable" + 0.028*"speedy" + 0.027*"tag" + 0.022*"note" + 0.021*"remove" + 0.021*"guidelines"

0.038*"list" + 0.018*"added" + 0.017*"references" + 0.015*"articles" + 0.015*"needs" + 0.012*"changes" + 0.012*"section" + 0.011*"reference" + 0.010*"add" + 0.010*"info"

0.027*"official" + 0.026*"youi" + 0.023*"children" + 0.018*"women" + 0.016*"names" + 0.015*"men" + 0.012*"happens" + 0.011*"drop" + 0.010*"born" + 0.010*"child"

0.048*"reverted" + 0.041*"

In [ ]:
for topic, p in lda.get_document_topics(training_BoW[1]):
    print(topics[topic])
    print()

In [ ]:
train_df.comment_text[0]

In [ ]:
train_df.comment_text[1]

In [107]:
dictionary.save('../lda_models/lda_1/dictionary.d')

In [108]:
lda.save('../lda_models/lda_1/lda.lda')

## Get LDA Features

In [104]:
from keras.preprocessing.sequence import pad_sequences

def get_lda_features(lda_model, texts, maxlen):    
    """
    Add one to the lda topic for embedding purposes as padding is 0 we do not want to 
    represent padding as a topic embedding
    """
    return pad_sequences(list(map(lambda text: 
                         list(map(lambda x: x[0] + 1,
                                  lda_model.get_document_topics(text))), texts)), maxlen=maxlen)
    

In [105]:
training_BoW = list(map(lambda text: dictionary.doc2bow(text), train_df.comment_text.astype(str).str.split()))
test_BoW = list(map(lambda text: dictionary.doc2bow(text), test_df.comment_text.astype(str).str.split()))

In [109]:
train_lda_matrix = get_lda_features(lda, training_BoW, 10)

In [110]:
test_lda_matrix = get_lda_features(lda, test_BoW, 10)

In [111]:
import numpy as np

np.save(open('../lda_features/train.npy', 'wb'), train_lda_matrix)
np.save(open('../lda_features/test.npy', 'wb'), test_lda_matrix)